## GEOSeMOSYS - geospatial energy systems optimization Benin

Nandi Moksnes, Will Usher

This Jupyter Notebook is both a description of how to reproduce the results in the paper and where you run the code.
For any questions please contact Nandi Moksnes: nandi@kth.se

## This repository is built from the version v0.1.0 on [GEOSeMOSYS_Kenya](https://github.com/KTH-dESA/GEOSeMOSYS_Kenya/releases/tag/v0.1.0-preprint)

## QGIS preparation

For the polygons to be matching the spatial resolution follwing steps are taken in QGIS to obtain the required point and polygon layers:
You need to have an (arbitrary) raster that is in projected metric CRS.
- r.resamp.stats and defined the cellsize to the sample size resolution (This function is available in QGIS GRASS)
- Then use that raster to generate: "Raster to point" and "Raster to polygon"

These files are then the base for running the global sensitivity analysis (GSA).

In [1]:
#Spatial resolution
point = '../Projected_files/13_points_BENIN_UTM31N.dbf.shp'
polygon = '../Projected_files/13_polygons_BENIN_UTM31N.shp'

In [2]:
%load_ext autoreload

from datetime import datetime
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns

# Initial key parameter settings

In "src/run/ref/input_data.csv", "src/run/vision/input_data.csv"key parameters are set for the following settings:
- Timeslice name
- Hour corresponding to each time slice (not used in the current version)
- Month
- Daysplit
- Specified annual demand fuel
- Start year
- End year
- Timeslice day-night hour (e.g. first hour that belongs to day time slice)
- Timeslice day-evening hour (e.g. last hour that belongs to day time slice)
- Timeslice evening-day hour (e.g. first hour that belongs to evening time slice)
- Timeslice evening-night hour (e.g. last hour that belongs to evening time slice)
- Timeslice night-evening hour (e.g. first hour that belongs to night time slice)
- Timeslice night-day hour (e.g. last hour that belongs to night time slice)
- Region (an OSeMOSYS set)
- Tech/renewable ninjafile equals the corresponding file to evaluate
- Household size (HH)
- Inhabited area (as a percentage)
- Area cell size (e.g. resolution on the data)
- peak(Watt) for households
- LV_area 
- Max capacity for LV lines in W


# GIS input files

In [3]:
#Benin
print(os.getcwd())
files = pd.read_csv('input_data/Benin_GIS_files.csv', index_col=0)
import os


crs = "EPSG:32631"
files

C:\Users\nandi\OneDrive - KTH\box_files\PhD\Paper 4 - GSA\reprod workflow\GSA_Spatial_temporal\src


,filename
type,
pop_raster,Benin/HRSL_Benin_1km_km_UTM31N.tif
adm_WGS84,gadm36_BEN_shp/gadm36_BEN_0.shp
adm,gadm36_BEN_0_UTM31N.shp
crs,UTM31N
minigrid_concat,Concat_Mini-grid_UMT31N.shp
MV_concat,Concat_MV_lines_UMT31N.shp
point,13_points_BENIN_UTM31N.dbf.shp
polygon,13_polygons_BENIN_UTM31N.shp
rastertopoint,Benin/raster_to_point_Benin_UTM31N.shp


## 1. The files in "input_data/GIS_data" are downloaded and placed in a "temp" folder.
Approx 25 min

### Datasets
[Nighttimelight data average masked, 2020](https://eogdata.mines.edu/nighttime_light/annual/v20/2020/VNL_v2_npp_2020_global_vcmslcfg_c202102150000.average_masked.tif.gz)
[GDP_PPP_2015_WGS_84](https://zenodo.org/record/4736323/files/GDP_PPP_Layer2015.tif?download=1)


In [4]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

from Download_files import *


URL_viirs = 'https://eogdata.mines.edu/nighttime_light/annual/v20/2020/VNL_v2_npp_2020_global_vcmslcfg_c202102150000.average_masked.tif.gz'
    
download_url_data('input_data/GIS_URL.txt', 'temp')
download_viirs(URL_viirs, 'temp')
unzip_all('input_data/GIS_unzip.txt','../temp', '../GIS_data')

2022 08 08-08:02:42_AM
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1091)>
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1091)>
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1091)>


## 2. The files are then projected and clipped to the administrative boundaries. 
Approx 3 hours

In [5]:
%autoreload
date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
print(date)
import os

from Project_GIS import *

project_main('../GIS_Data', '../Projected_files', files, crs)

2022_08_08-08:07:41_AM
C:\Users\nandi\OneDrive - KTH\box_files\PhD\Paper 4 - GSA\reprod workflow\GSA_Spatial_temporal\src
C:\Users\nandi\OneDrive - KTH\box_files\PhD\Paper 4 - GSA\reprod workflow\GSA_Spatial_temporal\GIS_Data
Already clipped


NameError: name 'input_schema' is not defined

## 3. Through QGIS make raster to point layer and save (MANUAL STEP)


1) Reproject the HRSL to CRS EPSG:32737 - WGS 84 / UTM zone 31N - Projected
2) Reduce the resolution through GRASS "r.resamp.stats" with cellsize 1000 m.
3) Remove values under 1 with Raster calculator.
4) Then run SAGA: raster to point in QGIS

This step is downloaded from zenodo so it is not needed for the user to execute.

Run time: Approx 1 minute


In [17]:
%load_ext autoreload
%autoreload

from datetime import datetime
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

from Download_files import *

#Make sure you are in the /src directory when you start this script
print(os.getcwd())

download_url_data("input_data/zenodo.txt", "Projected_files")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022 06 07-03:00:20_PM
C:\Users\nandi\OneDrive - KTH\box_files\PhD\Paper 4\reprod workflow\GSA_Spatial_temporal\GIS_Data


()

## 4. The GIS layers are prepared to for a heuristic approximation for electrified settlements
Approx 15 min

In [ ]:
%autoreload

from settlement_build import *

date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

os.chdir('../src')
#Make sure you are in the /src directory when you start this script
print(os.getcwd())

pop_shp = '../Projected_files/Benin/raster_to_point_Benin_UTM31N.shp'
Projected_files_path = '../Projected_files'

rasterize = raster_proximity(Projected_files_path, files)
points = raster_to_point(rasterize, pop_shp, Projected_files_path, crs)

## 5. Approximate location of urban settlements and the electrified settlements 1kmx1km resolution
Approx 10 min. Make sure to wait so that the shapefile and csv file is saved properly from step 4.

In [ ]:
%autoreload
from elec_start import *

date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

elec_actual = 0.403  # percent #https://data.worldbank.org/indicator/EG.ELC.ACCS.ZS accessed 2021-10-02
pop_cutoff = 400  # people
dist_mv = 1000 #meters
dist_lv = 1000 #meters
min_night_lights = 0.5
max_grid_dist = 5000  # meters
max_road_dist = 500  # meters
pop_cutoff2 = 3000 # people
urban_elec_ratio = 0.653  # percent https://data.worldbank.org/indicator/EG.ELC.ACCS.UR.ZS 2022-02-02
rural_elec_ratio = 0.174  # percent https://data.worldbank.org/indicator/EG.ELC.ACCS.RU.ZS accessed 2022-02-02
pop_actual = 12046162  # people https://data.worldbank.org/indicator/SP.POP.TOTL?locations=KE accessed 2021-10-02
urban = 0.48  # percent https://data.worldbank.org/indicator/SP.URB.TOTL.IN.ZS 2021-02-02
urban_cutoff = 20000
start_year = 2019
settlement = gpd.read_file("../Projected_files/settlements.shp")

settlements = pd.DataFrame(settlement, copy=True)
urbansettlements = calibrate_pop_and_urban(settlements, pop_actual, urban, urban_cutoff)
elec_current_and_future(urbansettlements, elec_actual, pop_cutoff, min_night_lights,
                            max_grid_dist, urban_elec_ratio, rural_elec_ratio, max_road_dist, pop_actual, pop_cutoff2, start_year, dist_mv, dist_lv, crs)


In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

shp_path = (r'..\Projected_files\elec.shp')

point = gpd.read_file(shp_path)
os.chdir('../src')

In [ ]:
fig, ax = plt.subplots(1, 1)
point.plot(column='elec', ax=ax)
fig.suptitle('Estimated electrified popluation (in yellow) Benin', fontsize=18)

## 6. Run Pathfinder for the area of study 
## Network can be overlapping due to that the split of attirbutes is not executed
- Build target.csv and weights.csv from unelectrified population per 1x1 km cell with a population over x and roadnetwork to weight 0.5 and where there is grid weight 0.01.
- Define the origin (center of the cell) and run the many-to-many Dijkstra [Pathfinder algorithm](https://github.com/facebookresearch/many-to-many-dijkstra) (Rohrer, 2019)
- The path where there is an overlap with grid is removed and the remainder is concidered the new distribution lines.
- The lenght of the path is summed to km and the length of the LV per cell is then multiplied with the cost per MW-km. [The cost per km is set to 4000 USD/MW-km and 15 000 USD/MW for a substation](https://iea-etsap.org/E-TechDS/PDF/E12_el-t&d_KV_Apr2014_GSOK.pdf).

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

#Identify unelectrified polygons

from Build_csv_files import *
polygons_all = '../Projected_files/Final_Polygons_Kenya.shp'
noHV = 'run/noHV_cells.csv'
shape =  "run/Demand/un_elec_polygons.shp"

noHV_polygons(polygons_all, noHV, shape)

# Scenario builder

## 1. Download Renewable.ninja data for all cells
To run this code you need to have installed R on your computer and the package "curl". In addition you need to have a log in on renewables.ninja and get your API token to dowload a higher share of locations per day

Approx 24 h due to the max dowloads per hour

## TODO: The shapefile needs to be flexible

In [9]:
%load_ext autoreload
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

from renewable_ninja_download import *

#To be able to download you need to install the package curl from R and also have R installed on your computer
# Easiest is to write  install.packages("curl") in the R prompt


# add your token for API from your own log in on Renewable Ninjas
token = ''
time_zone_offset = 1  # Benin is UTC + 1hours to adjust for the time zone

shapefile = '../Projected_files/' + files.loc['point','filename']
#Add the path to the RScript.exe under Program Files and add here
Rpath =  'C:\\TPFAPPS\\R\\R-4.1.0\\bin\\RScript.exe'
srcpath = os.getcwd()
print(srcpath)
path = "temp"
coordinates = project_vector(shapefile)
wind, solar = csv_make(coordinates)
down = download(path, Rpath, srcpath, wind, solar, token)
adjust_timezone(path, time_zone_offset)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022 02 24-04:23:59_PM
C:\Users\nandi\OneDrive - KTH\box_files\PhD\Paper 4\reprod workflow\GSA_Spatial_temporal\src


## 2. Scale down from1x1 km to chosens spatial resolution for electrified and unelectrified demand cells and save to un_elec/elec.csv and demand_cells.csv

From the demand_cell.csv file identify the cells that does not have any electrified cells in the 40x40 cell and classify them as unelectrified. 

Approx. 5 min

In [15]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

#Make sure you are in the /src directory when you start this script
print(os.getcwd())

from post_elec_GIS_functions import *
shape =  '../Projected_files/' + files.loc['polygon','filename']
gdp =  '../Projected_files/' + files.loc['gdp','filename']
elec_shp = '../Projected_files/elec.shp'
demandcells = os.path.join(os.getcwd(), 'run/Demand/demand_cells.csv')

join(elec_shp, gdp, shape)
elec(demandcells)


2022 02 24-04:32:58_PM
C:\Users\nandi\OneDrive - KTH\box_files\PhD\Paper 4\reprod workflow\GSA_Spatial_temporal\src
epsg:32631
EPSG:32631
UTM31N_GDP_PPP_2015_WGS_84


C:\Users\nandi\OneDrive - KTH\box_files\PhD\Paper 4\reprod workflow\GSA_Spatial_temporal\src\post_elec_GIS_functions.py:45: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  demand_cells.to_file(os.path.join(os.getcwd(), 'run\Demand\demand.shp'))


KeyError: 'Minigrid'

## 3. Calculate the demand per cell based on following equations (MANUAL STEP in Excel)


In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

from post_elec_GIS_functions import calculate_demand

settlements = 'run/Demand/demand_cells.csv'
demand = 'input_data/demand.csv'
calculate_demand(settlements, demand)


# 9. Distribution lines - Pathfinder



## 9.1 Identify the 40x40 km cells that has no HV line

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

#Identify unelectrified polygons

from Build_csv_files import *
polygons_all = '../Projected_files/Final_Polygons_Kenya.shp'
noHV = 'run/noHV_cells.csv'
shape =  "run/Demand/un_elec_polygons.shp"

noHV_polygons(polygons_all, noHV, shape)


## Manual step in ArcMap
- Split by attribute "Projected_files/Final_polygons_Kenya.shp" in attribute pointid to src/temp folder

This is dowloaded from Zenodo and not needed to execute.

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

from Download_files import *
import os
#Make sure you are in the /src directory when you start this script
print(os.getcwd())

download_url_data("input_data/dijkstra.txt","src/temp")
unzip_all('input_data/dijkstra.txt', 'temp')

## 9.2 Run Pathfinder for the area of study
- Build target.csv and weights.csv from unelectrified population per 1x1 km cell with a population over x and roadnetwork to weight 0.5 and where there is grid weight 0.01.
- Define the origin (center of the cell) and run the many-to-many Dijkstra [Pathfinder algorithm](https://github.com/facebookresearch/many-to-many-dijkstra) (Rohrer, 2019)
- The path where there is an overlap with grid is removed and the remainder is concidered the new distribution lines.
- The lenght of the path is summed to km and the length of the LV per cell is then multiplied with the cost per MW-km. [The cost per km is set to 4000 USD/MW-km and 15 000 USD/MW for a substation](https://iea-etsap.org/E-TechDS/PDF/E12_el-t&d_KV_Apr2014_GSOK.pdf).

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

from Pathfinder_processing_steps import *
path = '../Projected_files/'
proj_path = 'temp'
elec_shp = '../Projected_files/elec.shp'


pathfinder_main(path,proj_path, elec_shp)

## 10. Near table for all unelectrified cells (MANUAL STEP)

* In ArcMap Near ............within the distance 50 000 m.
* It is a simplification as it does not calculate the exact distance from the grid ut instead from the central point of the electrified cell.

## 10. Build the following csv files for building the data file to OSeMOSYS
- GIS_data.csv
- capacityfactor_solar.csv
- capacityfactor_wind.csv
- capitalcost.csv
- inputactivity.csv
- outputactivity.csv
- capacitytoactivity.csv

It also generates a technology and fuel file that is convenient to paste into the base OSeMOSYS txt file for the SETs TECHNOLOGY and FUEL.

## Transmission lines distance and presence of minigrid
The cells that has at least one electrified cell and are not situated 5000 m from a minigrid are concidered electrified by transmission lines.

Cost concidered are [2500 USD MW-km for transmission lines and 1500 USD/MW](https://iea-etsap.org/E-TechDS/PDF/E12_el-t&d_KV_Apr2014_GSOK.pdf) for substation and  [4000 USD/MW-km](https://iea-etsap.org/E-TechDS/PDF/E12_el-t&d_KV_Apr2014_GSOK.pdf) for low voltage lines.

The fixed cost of HV/LV transmission lines are set to 2.5% of the capital cost [Updated-Least-Cost-Power-Development-Plan-2017-2022](http://gak.co.ke/wp-content/uploads/2019/02/Updated-Least-Cost-Power-Development-Plan-2017-2022-min.pdf)



In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

from Build_csv_files import * 

renewable_path = 'temp'
pop_shp = 'new_40x40points_WGSUMT37S.shp'
unelec = 'run/un_elec.csv'
noHV = 'run/noHV_cells.csv'
HV = 'run/HV_cells.csv'
elec = 'run/elec.csv'
Projected_files_path = '../Projected_files/'
distribution_network = 'run/Demand/distributionlines.csv'
distribution_length_cell = 'run/Demand/Distribution_network.xlsx'

capital_cost_HV = 2.5 #kUSD MW-km
substation = 15 #kUSD/MW
capital_cost_LV = 4 #kUSD/MW
capital_cost_LV_strengthening = 1 #kUSD/MW Assumed 25% of the cost
capacitytoactivity = 31.536 #coversion MW to TJ
distribution_cost = '0'
path = 'run/ref'
neartable = 'run/Demand/Near_table.csv'

#Solar and wind csv files
renewableninja(renewable_path, path)
#Location file
GIS_file(path, files)
matrix = adjacency_matrix(neartable, noHV, HV, path)

capital_cost_transmission_distrib(capital_cost_LV_strengthening, distribution_network, elec, noHV, HV, unelec, capital_cost_HV, substation, capital_cost_LV, capacitytoactivity, distribution_cost, path, distribution_length_cell, matrix)

# 12. Update the final files that are not updated by the script (MANUAL step)

These files are automatically dowloaded for the required runs.

- Demandprofile.csv (KPLC, 2015 halfhourly profile)
- demandprofile_rural.csv [Williams et al., 2018](https://ieeexplore.ieee.org/document/8521099/)
- emissions.csv
- capitalcost_RET_ref.csv [NREL - moderate](https://atb.nrel.gov/)
- capitalcost_RET_vision.csv [NREL -  advanced](https://atb.nrel.gov/)
- Merge fixedcost_tnd to fixed_cost [NREL - moderate/advanced](https://atb.nrel.gov/)
- variable_cost.csv
- Operational_life.csv [NREL - moderate/advanced](https://atb.nrel.gov/)
- battery [NREL - moderate/advanced](https://atb.nrel.gov/)

# 14. Run the [GEOSeMOSYS code](https://github.com/KTH-dESA/GEOSeMOSYS) by Moksnes et al. (2020) through the command line
To fit the Kenya analysis the code is modified with the following additions:
- Electrified cells are treated separatley therefore an extra row is added to the functions where Solar PV, LV lines, or final electricity is calculated. They are treated as a separate cell with the extension of "_0" and "_1" for un-electrified and electrified

# 14 a) Reference scenario
Approx 3 h

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)


!python run/run_datafile_ref.py run/ref run/Kenya_basefile_reference.txt run/output/REF.txt

# 14 b) Vision 2030 scenario
Approx 3h

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

!python run/run_datafile_vision.py run/vision/update run/Vision.txt run/output/Vision.txt

# 14 c) Vision 2030 scenario dry hydro (can be rewritten to just replace the capacityfactors from Vision scenario)
Approx 3h

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

!python run/run_datafile_vision.py run/vision/update run/Dry_Vision.txt run/output/Dry_Vision.txt

# 15) Check the datafile before the run
The datafile can have Nan so those needs to be deleted as they cannot be processed in GNUMath Prog.

# 16) Run the data file on a High performance cluster
- This analysis was run on the KTH PDC resource Tegner with the performance of 2 x 12 cores Intel E5-2690v3 Haswell, 512 GB RAM 24 cores and 1  x NVIDIA Quadro K420. 

sbatch run_cplex_tegner_REF.sh

sbatch run_cplex_tegner_Vision.sh

sbatch run_cplex_tegner_DryHydro.sh

## 16) Extract dual values from the .sol file

In [ ]:
!python .\OSeMOSYS_CPLEX_dual_values.py run/output/REF.sol results/Kenya_REF_dual.txt
!python .\OSeMOSYS_CPLEX_dual_values.py run/output/Vision.sol results/Kenya_Vision_dual.txt
!python .\OSeMOSYS_CPLEX_dual_values.py run/output/Dry_Vision.sol results/Kenya_Dry_Vision_dual.txt

## 17) Results extraction
### To csv and pandas dataframes

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")

from Results_extraction import read_data

ref = read_data('results/REF_solved.txt')
vision = read_data('results/Vision_solved.txt')
dry_vision = read_data('results/Dry_vision_solved.txt')

prod_ref = ref['ProductionByTechnologyAnnual']
prod_vision = vision['ProductionByTechnologyAnnual']
prod_dry_vision = dry_vision['ProductionByTechnologyAnnual']

new_cap_ref = ref['NewCapacity']
new_cap_vision = vision['NewCapacity']
new_cap_dryvision = dry_vision['NewCapacity']

prod_ref.head(10)

### Make GIS files

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

from Build_csv_files import *
#Create maps for the results

polygons_all = '../Projected_files/Final_Polygons_Kenya.shp'

BaseyearHV = 'run/HV.csv'
shapeHV = 'results/connected.shp'

connectedHV_dryvision = 'results/connected_HV_dryvision.csv'
shapeHV_dry = 'results/connected_dry_vision.shp'

connectedHV_vision = 'results/connected_HV_vision.csv'
shapeHV_vision = 'results/connected_vision.shp'

connectedLV = 'results/connected_LV.csv'
shapeLV = 'results/connected_LV.shp'

connectedLV_vision = 'results/connected_LV_vision.csv'
shapeLV_vision = 'results/connected_LV_vision.shp'

distribution = 'results/electrified_distribution.csv'
shapedistribution = 'results/distribution.shp'

wind = 'results/wind.csv'
shapewind = 'results/wind_cells.shp'

wind_vision = 'results/wind_vision.csv'
shapewind_vision = 'results/wind_cells_vision.shp'

noHV_polygons(polygons_all, connectedHV, shapeHV)
noHV_polygons(polygons_all, connectedHV_dryvision, shapeHV)
noHV_polygons(polygons_all, connectedLV, shapeLV)
noHV_polygons(polygons_all, distribution, shapedistribution)
noHV_polygons(polygons_all, wind, shapewind)

noHV_polygons(polygons_all, connectedHV_vision, shapeHV_vision)
noHV_polygons(polygons_all, connectedLV_vision, shapeLV_vision)
noHV_polygons(polygons_all, wind_vision, shapewind_vision)


### Clustering analysis of the dual values based on capacityfactor of wind and extension of transmission line

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")

print(date)

from Results_extraction import clustering, normalize

datafile = pd.read_csv("results/cluster_dual.csv", index_col=0)

cols = ['norm2031','norm2025', 'normTRHV', 'normCF Wind'] #,'','SOMG4c','SOPV','SOPV4r','TRHV','WI','WI4c' KEEL00d

normalized_data = normalize(datafile)
data_for_clustering = np.array(normalized_data)

n_clusters = 3

df = clustering(n_clusters,data_for_clustering,cols[0],cols[1], cols[2], cols[3]) #,cols[2], cols[3],cols[4],cols[5], cols[6],cols[7],cols[8]

df.to_csv('cluster_results.csv')

join_data = df.append(datafile)

join_data


### Plot of clusters

In [ ]:
%matplotlib inline
sns.set_context("notebook",rc={"font.size": 18})
sns.set_style("whitegrid")

mycolors = ["grey","red","green","blue"]
for (i,subdf) in df.groupby("class"):

    plt.scatter(subdf['normCF Wind'],subdf['norm2031'],label="Cluster {}".format(i),c=mycolors[i-1])

plt.legend()
plt.xlabel('Capacityfactor wind')
plt.ylabel('Shadow price 2025')
plt.savefig("clusters.pdf")
